In [1]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
#GREEN_TAXI_DATA_2018_AUGUST

df_g = pd.read_csv("/Users/ranjani/taxidata/green_tripdata_2018-08.csv")
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,0.5,0.5,2.96,0.0,NaN,0.3,17.76,1,1
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,0.5,0.5,1.96,0.0,NaN,0.3,11.76,1,1
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2,1
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2,1
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,0.5,0.5,1.74,0.0,NaN,0.3,7.54,1,1


In [3]:
df_g.count()

VendorID                 666376
lpep_pickup_datetime     666376
lpep_dropoff_datetime    666376
store_and_fwd_flag       666376
RatecodeID               666376
PULocationID             666376
DOLocationID             666376
passenger_count          666376
trip_distance            666376
fare_amount              666376
extra                    666376
mta_tax                  666376
tip_amount               666376
tolls_amount             666376
ehail_fee                     0
improvement_surcharge    666376
total_amount             666376
payment_type             666376
trip_type                666376
dtype: int64

In [4]:
df_g['passenger_count'].isnull().sum()

0

In [3]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

In [4]:
sf = shapefile.Reader("shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
df_loc.head()

,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,zone,longitude,latitude
0,1,1,0.000782,0.116357,EWR,Newark Airport,9.366817e+05,190522.130278
1,2,2,0.004866,0.433470,Queens,Jamaica Bay,1.033536e+06,161853.982300
2,3,3,0.000314,0.084341,Bronx,Allerton/Pelham Gardens,1.027136e+06,254730.010849
3,4,4,0.000112,0.043567,Manhattan,Alphabet City,9.904240e+05,203100.040432
4,5,5,0.000498,0.092146,Staten Island,Arden Heights,9.321332e+05,139954.541936


In [5]:
df_g = df_g.join(df_loc.set_index('LocationID'),on=['PULocationID'])
df_g.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_g.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'}, inplace=True)
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,...,0.5,2.96,0.0,NaN,0.3,17.76,1,1,9.960899e+05,201154.900463
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,...,0.5,1.96,0.0,NaN,0.3,11.76,1,1,9.955246e+05,198307.925259
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,...,0.5,0.00,0.0,NaN,0.3,8.30,2,1,1.002507e+06,214131.084088
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,...,0.5,0.00,0.0,NaN,0.3,7.80,2,1,1.006325e+06,216569.411021
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,...,0.5,1.74,0.0,NaN,0.3,7.54,1,1,9.955246e+05,198307.925259


In [6]:
df_g['pickup_longitude'] = df_g['pickup_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_g['pickup_latitude'] = df_g['pickup_latitude'].apply(lambda x: '{:.2f}'.format(x))
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,...,0.5,2.96,0.0,NaN,0.3,17.76,1,1,996089.88,201154.90
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,...,0.5,1.96,0.0,NaN,0.3,11.76,1,1,995524.65,198307.93
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,...,0.5,0.00,0.0,NaN,0.3,8.30,2,1,1002506.87,214131.08
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,...,0.5,0.00,0.0,NaN,0.3,7.80,2,1,1006324.94,216569.41
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,...,0.5,1.74,0.0,NaN,0.3,7.54,1,1,995524.65,198307.93


In [7]:
df_g = df_g.join(df_loc.set_index('LocationID'),on=['DOLocationID'])
df_g.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,longitude,latitude
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,...,0.0,NaN,0.3,17.76,1,1,996089.88,201154.90,9.948706e+05,190214.741724
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,...,0.0,NaN,0.3,11.76,1,1,995524.65,198307.93,9.986127e+05,204798.780373
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,...,0.0,NaN,0.3,8.30,2,1,1002506.87,214131.08,1.006325e+06,216569.411021
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,...,0.0,NaN,0.3,7.80,2,1,1006324.94,216569.41,1.010063e+06,211298.529136
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,...,0.0,NaN,0.3,7.54,1,1,995524.65,198307.93,9.955246e+05,198307.925259


In [8]:
df_g.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude', 'lpep_pickup_datetime': 'pickup_datetime', 'lpep_dropoff_datetime':'dropoff_datetime'}, inplace=True)

In [9]:
df_g['dropoff_longitude'] = df_g['dropoff_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_g['dropoff_latitude'] = df_g['dropoff_latitude'].apply(lambda x: '{:.2f}'.format(x))
df_g.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,...,0.0,NaN,0.3,17.76,1,1,996089.88,201154.90,994870.57,190214.74
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,...,0.0,NaN,0.3,11.76,1,1,995524.65,198307.93,998612.72,204798.78
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,...,0.0,NaN,0.3,8.30,2,1,1002506.87,214131.08,1006324.94,216569.41
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,...,0.0,NaN,0.3,7.80,2,1,1006324.94,216569.41,1010063.10,211298.53
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,...,0.0,NaN,0.3,7.54,1,1,995524.65,198307.93,995524.65,198307.93


In [10]:
df_g.isnull().sum()

VendorID                      0
pickup_datetime               0
dropoff_datetime              0
store_and_fwd_flag            0
RatecodeID                    0
PULocationID                  0
DOLocationID                  0
passenger_count               0
trip_distance                 0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
ehail_fee                688357
improvement_surcharge         0
total_amount                  0
payment_type                  0
trip_type                     0
pickup_longitude              0
pickup_latitude               0
dropoff_longitude             0
dropoff_latitude              0
dtype: int64

In [11]:
df_g.dropna(subset=['pickup_datetime','dropoff_datetime','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2,2018-08-01 00:03:22,2018-08-01 00:17:59,N,1,255,49,1,3.54,13.5,...,0.0,NaN,0.3,17.76,1,1,996089.88,201154.90,994870.57,190214.74
1,2,2018-08-01 00:37:08,2018-08-01 00:46:53,N,1,256,112,1,1.94,8.5,...,0.0,NaN,0.3,11.76,1,1,995524.65,198307.93,998612.72,204798.78
2,2,2018-08-01 00:12:45,2018-08-01 00:20:21,N,1,146,7,2,1.37,7.0,...,0.0,NaN,0.3,8.30,2,1,1002506.87,214131.08,1006324.94,216569.41
3,2,2018-08-01 00:21:29,2018-08-01 00:27:41,N,1,7,260,2,1.10,6.5,...,0.0,NaN,0.3,7.80,2,1,1006324.94,216569.41,1010063.10,211298.53
4,2,2018-08-01 00:03:24,2018-08-01 00:06:44,N,1,256,256,1,0.56,4.5,...,0.0,NaN,0.3,7.54,1,1,995524.65,198307.93,995524.65,198307.93
5,2,2018-08-01 00:13:17,2018-08-01 00:32:57,N,1,80,186,2,5.34,18.5,...,0.0,NaN,0.3,23.76,1,1,1001683.72,199576.19,986346.34,211974.16
6,1,2018-08-01 00:08:52,2018-08-01 00:13:40,N,1,75,238,1,1.60,6.5,...,0.0,NaN,0.3,7.80,2,1,999364.85,227274.37,991717.63,227746.10
7,1,2018-08-01 00:57:43,2018-08-01 01:04:59,N,1,74,168,1,1.80,7.5,...,0.0,NaN,0.3,8.80,2,1,1001578.96,232789.87,1007873.14,233337.10
8,1,2018-08-01 00:49:49,2018-08-01 00:54:29,N,1,179,7,2,1.00,5.5,...,0.0,NaN,0.3,6.80,2,1,1004253.56,220334.19,1006324.94,216569.41
9,2,2018-08-01 00:44:06,2018-08-01 00:55:46,N,1,74,78,1,5.76,17.5,...,0.0,NaN,0.3,18.80,2,1,1001578.96,232789.87,1016891.42,247542.72


In [5]:
#YELLOW_TAXI_DATA_2018_AUGUST

df_y = pd.read_csv("/Users/ranjani/taxidata/yellow_tripdata_2018-08.csv")
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.6,1,N,238,79,1,19.0,0.5,0.5,4.00,0.0,0.3,24.30
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.9,1,N,138,7,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.4,1,N,231,7,1,30.0,0.5,0.5,7.80,0.0,0.3,39.10
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.7,1,N,79,148,1,5.5,0.5,0.5,0.00,0.0,0.3,6.80


In [6]:
df_y.count()

VendorID                 7849134
tpep_pickup_datetime     7849134
tpep_dropoff_datetime    7849134
passenger_count          7849134
trip_distance            7849134
RatecodeID               7849134
store_and_fwd_flag       7849134
PULocationID             7849134
DOLocationID             7849134
payment_type             7849134
fare_amount              7849134
extra                    7849134
mta_tax                  7849134
tip_amount               7849134
tolls_amount             7849134
improvement_surcharge    7849134
total_amount             7849134
dtype: int64

In [8]:
df_y.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [14]:
df_y = df_y.join(df_loc.set_index('LocationID'),on=['PULocationID'])
df_y.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_y.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'}, inplace=True)
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.6,1,N,238,79,1,19.0,0.5,0.5,4.00,0.0,0.3,24.30,9.917176e+05,227746.100420
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80,9.982076e+05,211280.687978
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.9,1,N,138,7,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35,1.019304e+06,221604.395797
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.4,1,N,231,7,1,30.0,0.5,0.5,7.80,0.0,0.3,39.10,9.822364e+05,201073.414111
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.7,1,N,79,148,1,5.5,0.5,0.5,0.00,0.0,0.3,6.80,9.883201e+05,204490.058251


In [15]:
df_y['pickup_longitude'] = df_y['pickup_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_y['pickup_latitude'] = df_y['pickup_latitude'].apply(lambda x: '{:.2f}'.format(x))
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.6,1,N,238,79,1,19.0,0.5,0.5,4.00,0.0,0.3,24.30,991717.63,227746.10
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.9,1,N,138,7,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35,1019304.02,221604.40
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.4,1,N,231,7,1,30.0,0.5,0.5,7.80,0.0,0.3,39.10,982236.37,201073.41
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.7,1,N,79,148,1,5.5,0.5,0.5,0.00,0.0,0.3,6.80,988320.12,204490.06


In [16]:
df_y = df_y.join(df_loc.set_index('LocationID'),on=['DOLocationID'])
df_y.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,longitude,latitude
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.6,1,N,238,79,1,...,0.5,0.5,4.00,0.0,0.3,24.30,991717.63,227746.10,9.883201e+05,204490.058251
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,9.982076e+05,211280.687978
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.9,1,N,138,7,1,...,0.5,0.5,2.55,0.0,0.3,15.35,1019304.02,221604.40,1.006325e+06,216569.411021
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.4,1,N,231,7,1,...,0.5,0.5,7.80,0.0,0.3,39.10,982236.37,201073.41,1.006325e+06,216569.411021
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.7,1,N,79,148,1,...,0.5,0.5,0.00,0.0,0.3,6.80,988320.12,204490.06,9.868379e+05,201264.700017


In [17]:
df_y.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude', 'tpep_pickup_datetime': 'pickup_datetime', 'tpep_dropoff_datetime':'dropoff_datetime'}, inplace=True)

In [18]:
df_y['dropoff_longitude'] = df_y['dropoff_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_y['dropoff_latitude'] = df_y['dropoff_latitude'].apply(lambda x: '{:.2f}'.format(x))
df_y.head()

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.6,1,N,238,79,1,...,0.5,0.5,4.00,0.0,0.3,24.30,991717.63,227746.10,988320.12,204490.06
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.57,211280.69
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.9,1,N,138,7,1,...,0.5,0.5,2.55,0.0,0.3,15.35,1019304.02,221604.40,1006324.94,216569.41
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.4,1,N,231,7,1,...,0.5,0.5,7.80,0.0,0.3,39.10,982236.37,201073.41,1006324.94,216569.41
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.7,1,N,79,148,1,...,0.5,0.5,0.00,0.0,0.3,6.80,988320.12,204490.06,986837.85,201264.70


In [19]:
df_y.isnull().sum()

VendorID                 0
pickup_datetime          0
dropoff_datetime         0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
dtype: int64

In [20]:
df_y.dropna(subset=['pickup_datetime','dropoff_datetime','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2018-08-01 00:44:35,2018-08-01 01:03:22,1,5.60,1,N,238,79,1,...,0.5,0.5,4.00,0.00,0.3,24.30,991717.63,227746.10,988320.12,204490.06
1,1,2018-08-01 00:02:19,2018-08-01 00:02:31,1,0.00,1,N,145,145,2,...,0.5,0.5,0.00,0.00,0.3,3.80,998207.57,211280.69,998207.57,211280.69
2,1,2018-08-01 00:13:25,2018-08-01 00:24:40,1,2.90,1,N,138,7,1,...,0.5,0.5,2.55,0.00,0.3,15.35,1019304.02,221604.40,1006324.94,216569.41
3,1,2018-08-01 00:10:37,2018-08-01 00:49:10,1,8.40,1,N,231,7,1,...,0.5,0.5,7.80,0.00,0.3,39.10,982236.37,201073.41,1006324.94,216569.41
4,1,2018-08-01 00:02:18,2018-08-01 00:07:32,2,0.70,1,N,79,148,1,...,0.5,0.5,0.00,0.00,0.3,6.80,988320.12,204490.06,986837.85,201264.70
5,1,2018-08-01 00:22:51,2018-08-01 00:55:23,2,6.70,1,N,114,7,1,...,0.5,0.5,4.02,0.00,0.3,30.82,985030.85,204700.77,1006324.94,216569.41
6,1,2018-08-01 00:02:52,2018-08-01 00:15:50,1,4.50,1,N,132,205,2,...,0.5,0.5,0.00,0.00,0.3,15.80,1042627.91,173494.35,1049852.90,191369.27
7,1,2018-08-01 00:57:29,2018-08-01 01:26:37,1,16.80,2,N,132,162,1,...,0.0,0.5,14.60,5.76,0.3,73.16,1042627.91,173494.35,991876.75,215015.15
8,2,2018-08-01 00:36:51,2018-08-01 00:38:35,5,0.43,1,N,88,87,1,...,0.5,0.5,1.20,0.00,0.3,6.00,981197.09,195500.41,982242.97,196731.42
9,1,2018-08-01 00:29:32,2018-08-01 00:54:36,1,5.00,1,N,234,7,1,...,0.5,0.5,5.20,0.00,0.3,26.00,986894.30,208964.96,1006324.94,216569.41


In [21]:
df_g.to_csv("green_tripdata_2018_aug_output.csv", index=False)

In [22]:
df_y.to_csv("yellow_tripdata_2018_aug_output.csv", index=False)